<a href="https://colab.research.google.com/github/Shailendrakulha/MachineLearning_Algo/blob/main/VGG16ChestX_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten

In [5]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [25]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'

In [26]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [27]:
for layer in vgg.layers:
    layer.trainable = False

In [28]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')
x = Flatten()(vgg.output)

In [29]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [30]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [31]:
from keras.preprocessing.image import ImageDataGenerator

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [34]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
522/522 [==============================] - 471s 818ms/step - loss: 0.2106 - accuracy: 0.9285 - val_loss: 0.3090 - val_accuracy: 0.9119
Epoch 2/5
522/522 [==============================] - 141s 270ms/step - loss: 0.1664 - accuracy: 0.9492 - val_loss: 0.2830 - val_accuracy: 0.9215
Epoch 3/5
522/522 [==============================] - 138s 265ms/step - loss: 0.1068 - accuracy: 0.9647 - val_loss: 0.4124 - val_accuracy: 0.9038
Epoch 4/5
522/522 [==============================] - 140s 269ms/step - loss: 0.1513 - accuracy: 0.9572 - val_loss: 0.6119 - val_accuracy: 0.8846
Epoch 5/5
522/522 [==============================] - 142s 272ms/step - loss: 0.1402 - accuracy: 0.9601 - val_loss: 0.5455 - val_accuracy: 0.8974


In [35]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')

In [36]:
from keras.models import load_model

In [37]:
from tensorflow.keras.preprocessing import image

In [38]:
from keras.applications.vgg16 import preprocess_input

In [39]:
import numpy as np

In [40]:
model=load_model('chest_xray.h5')

In [42]:
img=image.load_img(r'/content/drive/MyDrive/chest_xray/val/PNEUMONIA/VIRUS (1).jpeg' ,target_size=(224,224))

In [43]:
x=image.img_to_array(img)

In [44]:
x=np.expand_dims(x, axis=0)

In [45]:
img_data=preprocess_input(x)

In [46]:
classes=model.predict(img_data)

1/1 [==============================] - 1s 825ms/step


In [47]:
result=int(classes[0][0])

In [48]:
if result==0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")

Person is Affected By PNEUMONIA


In [ ]:
# Model saving on drive

In [49]:
path = '/content/drive/MyDrive/chest_xray.h5'


model.save(path)

In [50]:
model_deploy = keras.models.load_model('chest_xray.h5')

In [52]:
model_deploy.predict(test_set).round(2)

63/63 [==============================] - 8s 117ms/step


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)